In [2]:
import os
import re
import numpy as np
import tensorflow as tf
import time
import sys
from functions import *

print("hello :)")


hello :)


In [3]:

base_dir = "keras/content/keras-facenet"
sys.path.append(base_dir + '/code/')
from inception_resnet_v1 import *

tf_model_dir = base_dir + '/model/20180402-114759/'
npy_weights_dir = base_dir + '/model/keras/npy_weights/'
weights_dir = base_dir + '/model/keras/weights/'
model_dir = base_dir + '/model/keras/model/'

weights_filename = 'facenet_keras_weights.h5'
model_filename = 'facenet_keras.h5'
model_filename_new = "facenet_keras.keras"
model_filename_tfl = "facenet_keras.tflite"
tfl_default_optimized_v1 = "tf1_default_optimized_v1.tflite"
float_16_optimized = "faceNet_16.tflite"

if not os.path.exists(npy_weights_dir):
    os.makedirs(npy_weights_dir, exist_ok=True)
if not os.path.exists(npy_weights_dir):
    os.makedirs(weights_dir, exist_ok=True)
if not os.path.exists(npy_weights_dir):
    os.makedirs(model_dir, exist_ok=True)



In [4]:
# regex for renaming the tensors to their corresponding Keras counterpart
re_repeat = re.compile(r'Repeat_[0-9_]*b')
re_block8 = re.compile(r'Block8_[A-Za-z]')

def get_filename(key):
    filename = str(key)
    filename = filename.replace('/', '_')
    filename = filename.replace('InceptionResnetV1_', '')

    # remove "Repeat" scope from filename
    filename = re_repeat.sub('B', filename)

    if re_block8.match(filename):
        # the last block8 has different name with the previous 5 occurrences
        filename = filename.replace('Block8', 'Block8_6')

    # from TF to Keras naming
    filename = filename.replace('_weights', '_kernel')
    filename = filename.replace('_biases', '_bias')

    return filename + '.npy'


def extract_tensors_from_checkpoint_file(filename, output_folder):
    reader = tf.compat.v1.train.NewCheckpointReader(filename)

    for key in reader.get_variable_to_shape_map():
        # not saving the following tensors
        if key == 'global_step':
            continue
        if 'AuxLogit' in key:
            continue

        # convert tensor name into the corresponding Keras layer weight name and save
        path = os.path.join(output_folder, get_filename(key))
        arr = reader.get_tensor(key)
        np.save(path, arr)

In [8]:
extract_tensors_from_checkpoint_file(tf_model_dir+'model-20180402-114759.ckpt-275', npy_weights_dir)

In [4]:
model = InceptionResNetV1(classes=512)
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 160, 160, 3)]        0         []                            
                                                                                                  
 Conv2d_1a_3x3 (Conv2D)      (None, 79, 79, 32)           864       ['input_1[0][0]']             
                                                                                                  
 Conv2d_1a_3x3_BatchNorm (B  (None, 79, 79, 32)           96        ['Conv2d_1a_3x3[0][0]']       
 atchNormalization)                                                                               
                                                                                                  
 Conv2d_1a_3x3_Activation (  (None, 79, 79, 32)           0         ['Conv2d_1a_

In [10]:
print('Loading numpy weights from', npy_weights_dir)
for layer in model.layers:
    if layer.weights:
        weights = []
        for w in layer.weights:
            print(w.name)
            weight_name = os.path.basename(w.name).replace(':0', '')
            weight_file = layer.name + '_' + weight_name + '.npy'
            weight_arr = np.load(os.path.join(npy_weights_dir, weight_file))
            weights.append(weight_arr)
        layer.set_weights(weights)


Loading numpy weights from keras/content/keras-facenet/model/keras/npy_weights/
Conv2d_1a_3x3/kernel:0
Conv2d_1a_3x3_BatchNorm/beta:0
Conv2d_1a_3x3_BatchNorm/moving_mean:0
Conv2d_1a_3x3_BatchNorm/moving_variance:0
Conv2d_2a_3x3/kernel:0
Conv2d_2a_3x3_BatchNorm/beta:0
Conv2d_2a_3x3_BatchNorm/moving_mean:0
Conv2d_2a_3x3_BatchNorm/moving_variance:0
Conv2d_2b_3x3/kernel:0
Conv2d_2b_3x3_BatchNorm/beta:0
Conv2d_2b_3x3_BatchNorm/moving_mean:0
Conv2d_2b_3x3_BatchNorm/moving_variance:0
Conv2d_3b_1x1/kernel:0
Conv2d_3b_1x1_BatchNorm/beta:0
Conv2d_3b_1x1_BatchNorm/moving_mean:0
Conv2d_3b_1x1_BatchNorm/moving_variance:0
Conv2d_4a_3x3/kernel:0
Conv2d_4a_3x3_BatchNorm/beta:0
Conv2d_4a_3x3_BatchNorm/moving_mean:0
Conv2d_4a_3x3_BatchNorm/moving_variance:0
Conv2d_4b_3x3/kernel:0
Conv2d_4b_3x3_BatchNorm/beta:0
Conv2d_4b_3x3_BatchNorm/moving_mean:0
Conv2d_4b_3x3_BatchNorm/moving_variance:0
Block35_1_Branch_2_Conv2d_0a_1x1/kernel:0
Block35_1_Branch_2_Conv2d_0a_1x1_BatchNorm/beta:0
Block35_1_Branch_2_Conv2

e:\taha\code\FaceNet-tfLite\.venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


also .keras format...


In [11]:

print('Saving weights...')
model.save_weights(os.path.join(weights_dir, weights_filename))
print('Saving model...')
model.save(os.path.join(model_dir, model_filename))
print("also .keras format...")
model.save(os.path.join(model_dir, model_filename_new))

Saving weights...
Saving model...
also .keras format...


In [3]:
model = tf.keras.models.load_model('..\\src\\keras\\content\\keras-facenet\\model\\keras\\model\\facenet_keras.h5', compile=False)

e:\taha\code\ras-face\.venv\lib\site-packages\keras\src\layers\core\lambda_layer.py:327: UserWarning: inception_resnet_v1 is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


##### basic convert model

In [13]:

# Convert the model. 
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open(os.path.join(model_dir, model_filename_tfl), 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\siedt\AppData\Local\Temp\tmpg6yhejm8\assets


INFO:tensorflow:Assets written to: C:\Users\siedt\AppData\Local\Temp\tmpg6yhejm8\assets


##### default optimization

In [3]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model.
with open(os.path.join(model_dir, tfl_default_optimized_v1), 'wb') as f:
  f.write(tflite_model)

NameError: name 'model' is not defined

##### float_16 optimization

In [5]:
# del converter
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()

# Save the model.
with open(os.path.join(model_dir, float_16_optimized), 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\siedt\AppData\Local\Temp\tmp6fdmkhqs\assets


INFO:tensorflow:Assets written to: C:\Users\siedt\AppData\Local\Temp\tmp6fdmkhqs\assets


In [4]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=os.path.join(model_dir, model_filename_tfl))
interpreter.allocate_tensors()



In [5]:
def predict(model, samples):
    input = model.get_input_details()
    output = model.get_output_details()

    print(f"input: {input}")
    print(f"output: {output}")

    # Test the model on random input data.
    input_shape = input[0]['shape']
    # print(f"shape: {input_shape}")

    #input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
    outputs = []
    for sample in samples:
        # print(f"sample shape: {sample.shape}")
        input_data = sample.reshape(input_shape)
        print(f"in  shape: {input_data.shape}")
        
        #input_data = np.expand_dims(input_data, axis=0)
        st = time.time()
        model.set_tensor(np.float32(input[0]['index']), input_data.astype('float32'))
        model.invoke()
        end = time.time()
        print(f"duration: {end-st} s") 

        # The function `get_tensor()` returns a copy of the tensor data.
        # Use `tensor()` in order to get a pointer to the tensor.
        output_data = model.get_tensor(output[0]['index'])
        #print(output_data)
        outputs.append(output_data)
    ret = np.stack(outputs)
    return ret

import cv2
from PIL import Image
def read_image(filename):

    # load image from file
    image = Image.open(filename)

    # convert to RGB, if needed
    image = image.convert('RGB')

    # convert to array
    pixels = np.asarray(image)

    return pixels


In [7]:
taha2 = read_image("dataset/apa-dataset/crop/train/taha/photo_2023-10-11_16-48-57-cropped.jpg")
taha1 = read_image("dataset/apa-dataset/crop/train/taha/photo_2023-10-11_16-49-25-cropped.jpg")
ali1 = read_image("dataset/apa-dataset/crop/train/Ali_Hajghani/photo_10_2023-10-11_16-44-44-cf.jpg")
ali2 = read_image("dataset/apa-dataset/crop/train/Ali_Hajghani/photo_10_2023-10-11_16-44-44-crr.jpg")
moslem1 = read_image('dataset/apa-dataset/crop/train/Moslem_attar/photo_3_2023-10-12_16-14-35-cropped.jpg')
moslem2 = read_image('dataset/apa-dataset/crop/train/Moslem_attar/photo_3_2023-10-12_16-14-35-cf.jpg')
mohammad1 = read_image("dataset/apa-dataset/crop/train/Mohammad_mohammadi/photo_2023-10-11_17-08-50-cropped.jpg")
mohammad2 = read_image("dataset/apa-dataset/crop/train/Mohammad_mohammadi/photo_2023-10-11_17-08-50-cfn.jpg")

imgs = [taha1,taha2,ali1,ali2,moslem1,moslem2,mohammad1,mohammad2]

# preprocess
input_imgs = []
for i in imgs:
     # standardize pixel values across channels (global)
    mean, std = i.mean(), i.std()
    i = (i - mean) / std
    print(mean, std)
    input_imgs.append(i.reshape((1, 160, 160, 3)))

# Create arrays to store the results
_predictions = predict(interpreter, input_imgs)

print("eucd distance taha1 vs taha2", np.linalg.norm(_predictions[0, :].flatten() - _predictions[1, :].flatten()))
print("eucd distance ali1 vs ali2", np.linalg.norm(_predictions[2, :].flatten() - _predictions[3, :].flatten()))
print("eucd distance ali1 vs taha1", np.linalg.norm(_predictions[2, :] - _predictions[0, :]))
print("eucd distance moslem1 vs moslem2", np.linalg.norm(_predictions[4, :].flatten() - _predictions[5, :].flatten()))
print("eucd distance mohammad1 vs mohammad2", np.linalg.norm(_predictions[6, :].flatten() - _predictions[7, :].flatten()))

from scipy.spatial import distance
print(_predictions[0, :].flatten().shape)

print("cosine distance taha1 vs taha2", distance.cosine(_predictions[0, :].flatten() , _predictions[1, :].flatten()))
print("cosine distance ali1 vs ali2", distance.cosine(_predictions[2, :].flatten() , _predictions[3, :].flatten()))
print("cosine distance ali1 vs taha1", distance.cosine(_predictions[2, :].flatten() , _predictions[0, :].flatten()))
print("cosine distance moslem1 vs moslem2", distance.cosine(_predictions[4, :].flatten() , _predictions[5, :].flatten()))
print("cosine distance mohammad1 vs mohammad2", distance.cosine(_predictions[6, :].flatten() , _predictions[7, :].flatten()))


118.50266927083334 65.38129107047106
110.35364583333333 56.33430221898169
105.163203125 41.87791818377146
94.71079427083333 53.11718023962414
82.34104166666667 33.756938221373055
82.35462239583333 33.76632106506235
120.13216145833333 46.79375550415103
120.3832421875 49.58518147370126
input: [{'name': 'serving_default_input_1:0', 'index': 0, 'shape': array([  1, 160, 160,   3]), 'shape_signature': array([ -1, 160, 160,   3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
output: [{'name': 'StatefulPartitionedCall:0', 'index': 448, 'shape': array([  1, 512]), 'shape_signature': array([ -1, 512]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
in  shap

In [8]:

data = _predictions[2, :].flatten()
label = "ali_haj"
save_to_csv("database.csv", data, label)

# Example usage:
data = _predictions[0, :].flatten()
label = "taha"
save_to_csv("database.csv", data, label)

data = _predictions[4, :].flatten()
label = "moslem"
save_to_csv("database.csv", data, label)

data = _predictions[6, :].flatten()
label = "mohammad"
save_to_csv("database.csv", data, label)


# Example usage:
new_array = _predictions[7, :].flatten()
most_similar_label, max_similarity = find_max_cosine_similarity("database.csv", new_array)
print(f"Most similar label: {most_similar_label}, Cosine Similarity: {max_similarity}")

Most similar label: mohammad, Cosine Similarity: 0.7901896910517987


In [12]:
keras_model = tf.keras.models.load_model(os.path.join(model_dir, model_filename_new), safe_mode=False)


In [13]:
# print(imgs[0].shape)
k_preds = []
for i,inp in enumerate(input_imgs):
    st = time.time()
    k_preds.append(keras_model.predict(input_imgs[i]))
    end = time.time()
    print(f"duration: {end-st}")
    # print(k_preds[i])
res = np.stack(k_preds)
print("distance taha1 vs taha2", np.linalg.norm(res[0, :] - res[1, :]))

1/1 [==============================] - 2s 2s/step
duration: 1.9458434581756592
1/1 [==============================] - 0s 60ms/step
duration: 0.11054253578186035
1/1 [==============================] - 0s 67ms/step
duration: 0.11224842071533203
1/1 [==============================] - 0s 68ms/step
duration: 0.13199758529663086
distance taha1 vs taha2 62.379707
